In [6]:
from collections import defaultdict
from itertools import chain
from pathlib import Path
from pprint import pprint
from uuid import uuid4
import json
import os
import re
import requests
import shutil

import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from schematics.types import IntType, StringType, ListType

%cd /home/ubuntu/hidebound/python
import hidebound.core.tools as tools
from hidebound.core.parser import AssetNameParser
from hidebound.core.database import Database
import hidebound.core.database_tools as db_tools
from hidebound.core.database_test import DatabaseTests
from hidebound.core.specification_base import *
from hidebound.core.specifications import *
from hidebound.core.validators import *
from hidebound.core.traits import *
import hidebound.server.components as components

/home/ubuntu/hidebound/python


In [8]:
class Spec001(SequenceSpecificationBase):
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'coordinate', 'frame', 'extension']
    descriptor = ListType(StringType(), required=True, validators=[is_descriptor, is_homogenous])
    frame = ListType(IntType(), required=True, validators=[is_frame])
    extension = ListType(StringType(), required=True, validators=[is_extension, lambda x: is_eq(x, 'png')])
    coordinate = ListType(ListType(IntType(), required=True, validators=[is_coordinate]))
#     file_traits = dict(
#         width=get_image_width
#     )

class Spec002(SequenceSpecificationBase):
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'frame', 'extension']
    width = ListType(IntType(), required=True, validators=[lambda x: is_eq(x, 1024)])
    height = ListType(IntType(), required=True, validators=[lambda x: is_eq(x, 1024)])
    frame = ListType(IntType(), required=True, validators=[is_frame, lambda x: is_gt(x, -1)])
    extension = ListType(StringType(), required=True, validators=[is_extension, lambda x: is_eq(x, 'exr')])
    file_traits = dict(
        width=get_image_width
    )
    
class Vdb001(FileSpecificationBase):
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'extension']
    extension = ListType(StringType(), required=True, validators=[is_extension, lambda x: is_eq(x, 'vdb')])

# db = Database(root, [Spec001, Spec002, Vdb001])
# db = Database(root, [Spec001, Spec002], exclude_regex='misc\.txt|vdb001')
root = '/tmp/projects'
hb_root = '/tmp/hidebound'
os.makedirs(hb_root, exist_ok=True)
# db = Database(root, hb_root, [Spec001, Spec002, Vdb001])
db = Database(root, hb_root, [Vdb001])
db.update()
db.data.T
db.update().create()
# db.delete()

!tree /tmp/hidebound/; tree /tmp/projects

/tmp/hidebound/
├── data
│   └── proj002
│       └── vdb001
│           ├── p-proj002_s-vdb001_d-bagel_v001.vdb
│           ├── p-proj002_s-vdb001_d-bagel_v002.vdb
│           └── p-proj002_s-vdb001_d-bagel_v003.vdb
├── hidebound_config.json
├── metadata
│   ├── asset
│   │   ├── 2763cdec-939e-454a-bba0-77cdadb9d8ce.json
│   │   ├── 29e5dea9-06cd-4e78-b597-633baf826688.json
│   │   └── 889f6954-a241-4565-8a3a-81a345f704d9.json
│   └── file
│       ├── 18133fab-29d1-450a-9f3d-5c735eee86ef.json
│       ├── a6c2535c-a834-47c4-b979-3f0997493e0e.json
│       └── baef9d38-175b-41f9-9f6d-b25dfc65fada.json
└── specifications

7 directories, 10 files
/tmp/projects
├── proj001
│   ├── spec001
│   │   └── pizza
│   │       ├── p-proj001_s-spec001_d-pizza_v001
│   │       │   ├── p-proj001_s-spec001_d-pizza_v001_c000-001_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v001_c000-001_f0002.png
│   │       │   └── p-proj001_s-spec001_d-pizza_v001_c000-001_f0003.png
│   │       ├── p-proj001

In [14]:
db.search('SELECT filename, asset_valid FROM data WHERE specification == "vdb001"')

filename  asset_valid
0  p-proj002_s-vdb001_d-bagel_v001.vdb            1
1  p-proj002_s-vdb001_d-bagel_v002.vdb            1
2  p-proj002_s-vdb001_d-bagel_v003.vdb            1

In [5]:
root = '/tmp/projects'
if Path(root).exists():
    shutil.rmtree(root)
os.makedirs(root)
fullpaths = DatabaseTests().create_files(root)

hb_root = '/tmp/hidebound'
if Path(hb_root).exists():
    shutil.rmtree(hb_root)
os.makedirs(hb_root)

/home/ubuntu/hidebound/python/hidebound/core/database_test_base.py:98: UserWarning: /tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v001/p-proj001_s-spec001_d-pizza_v001_c000-001_f0001.png is a low contrast image
  skimage.io.imsave(filepath.as_posix(), img)
/home/ubuntu/hidebound/python/hidebound/core/database_test_base.py:98: UserWarning: /tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v001/p-proj001_s-spec001_d-pizza_v001_c000-001_f0002.png is a low contrast image
  skimage.io.imsave(filepath.as_posix(), img)
/home/ubuntu/hidebound/python/hidebound/core/database_test_base.py:98: UserWarning: /tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v001/p-proj001_s-spec001_d-pizza_v001_c000-001_f0003.png is a low contrast image
  skimage.io.imsave(filepath.as_posix(), img)
/home/ubuntu/hidebound/python/hidebound/core/database_test_base.py:98: UserWarning: /tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v002/p-proj001_s-spec001_d-

In [3]:
!tree /tmp/projects/; tree /tmp/hidebound/

/tmp/projects/
├── proj001
│   ├── spec001
│   │   └── pizza
│   │       ├── p-proj001_s-spec001_d-pizza_v001
│   │       │   ├── p-proj001_s-spec001_d-pizza_v001_c000-001_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v001_c000-001_f0002.png
│   │       │   └── p-proj001_s-spec001_d-pizza_v001_c000-001_f0003.png
│   │       ├── p-proj001_s-spec001_d-pizza_v002
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0002.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0003.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0004.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0002.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0003.png
│   │       │   └── p-proj001_s-spec001_d-pizza_v002_c000-001_f0004.png
│   │       └── p-proj001_s-spec001_d-pizza_v003


In [2]:
url = 'http://0.0.0.0:80/api/initialize'
config = dict(
    root_directory='/tmp/projects',
    hidebound_directory='/tmp/hidebound',
    specification_files=['/home/ubuntu/hidebound/python/hidebound/core/test_specifications.py']
)
# config = dict(
#     root_directory='/mnt/storage/projects',
#     hidebound_directory='/mnt/storage/hidebound',
#     specification_files=['/home/ubuntu/hidebound/python/hidebound/core/test_specifications.py']
# )
config = json.dumps(config)
response = requests.post(url, json=config).json()['message']
print(response)

url = 'http://0.0.0.0:80/api/update'
response = requests.post(url).json()['message']

print(response)

url = 'http://0.0.0.0:80/api/read'
response = requests.post(url).json()['response']
data = response
print(response[0])

url = 'http://0.0.0.0:80/api/search'
params = json.dumps({'query': 'SELECT * FROM data WHERE asset_valid'})
response = requests.post(url, json=params).json()['response']
data = response
# DataFrame(response).T

# url = 'http://0.0.0.0:80/api/search'
# query = json.dumps({'query': 'SELECT * FROM data WHERE version == 3'})
# response = requests.post(url, json=query).json()['response']

# url = 'http://0.0.0.0:80/api/create'
# response = requests.post(url).json()['message']
# print(response)

Database initialized.
Database updated.
{'project': 'proj001', 'specification': 'spec001', 'descriptor': 'pizza', 'version': 1.0, 'coordinate_x': 0.0, 'coordinate_y': 1.0, 'frame': 1.0, 'extension': 'png', 'filename': 'p-proj001_s-spec001_d-pizza_v001_c000-001_f0001.png', 'filepath': '/tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v001/p-proj001_s-spec001_d-pizza_v001_c000-001_f0001.png', 'file_error': None, 'asset_name': 'p-proj001_s-spec001_d-pizza_v001', 'asset_path': '/tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v001', 'asset_type': 'sequence', 'asset_error': None, 'asset_valid': True, 'filename_error': None, 'width': None, 'width_error': None}


In [6]:
!tree /tmp/hidebound/; tree /tmp/projects

/tmp/hidebound/

0 directories, 0 files
/tmp/projects
├── proj001
│   ├── spec001
│   │   └── pizza
│   │       ├── p-proj001_s-spec001_d-pizza_v001
│   │       │   ├── p-proj001_s-spec001_d-pizza_v001_c000-001_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v001_c000-001_f0002.png
│   │       │   └── p-proj001_s-spec001_d-pizza_v001_c000-001_f0003.png
│   │       ├── p-proj001_s-spec001_d-pizza_v002
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0002.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0003.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0004.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0002.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0003.png
│   │       │   └── p-proj001_s-spec001_d-pizza_v002_c000-001_f0004.png
│   │     

In [2]:
class Spec001(SequenceSpecificationBase):
    name = 'spec001'
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'coordinate', 'frame', 'extension']
    descriptor = ListType(StringType(), required=True, validators=[is_descriptor, is_homogenous])
    frame = ListType(IntType(), required=True, validators=[is_frame])
    extension = ListType(StringType(), required=True, validators=[is_extension, lambda x: is_eq(x, 'png')])
    coordinate = ListType(ListType(IntType(), required=True, validators=[is_coordinate]))
#     file_traits = dict(
#         width=get_image_width
#     )

class Spec002(SequenceSpecificationBase):
    name = 'spec002'
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'frame', 'extension']
    width = ListType(IntType(), required=True, validators=[lambda x: is_eq(x, 1024)])
    height = ListType(IntType(), required=True, validators=[lambda x: is_eq(x, 1024)])
    frame = ListType(IntType(), required=True, validators=[is_frame, lambda x: is_gt(x, -1)])
    extension = ListType(StringType(), required=True, validators=[is_extension, lambda x: is_eq(x, 'exr')])
    file_traits = dict(
        width=get_image_width
    )
    
class Vdb001(FileSpecificationBase):
    name = 'vdb001'
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'extension']
    extension = StringType(required=True, validators=[is_extension, lambda x: is_eq(x, 'vdb')])

# db = Database(root, [Spec001, Spec002, Vdb001])
# db = Database(root, [Spec001, Spec002], exclude_regex='misc\.txt|vdb001')
root = '/tmp/projects'
hb_root = '/tmp/hidebound'
os.makedirs(hb_root, exist_ok=True)
db = Database(root, hb_root, [Spec001, Spec002])
db.update()
db.data.T
# db.update().create()

# !tree /tmp/hidebound/; tree /tmp/projects

0   \
specification                                                  spec001   
extension                                                          png   
filename             p-proj001_s-spec001_d-pizza_v001_c000-001_f000...   
filepath             /tmp/projects/proj001/spec001/pizza/p-proj001_...   
file_error                                                         NaN   
file_traits          {'project': 'proj001', 'specification': 'spec0...   
asset_name                            p-proj001_s-spec001_d-pizza_v001   
asset_path           /tmp/projects/proj001/spec001/pizza/p-proj001_...   
asset_type                                                    sequence   
asset_traits         {'project': ['proj001', 'proj001', 'proj001'],...   
asset_error                                                        NaN   
asset_valid                                                       True   
asset_path_relative                                     <class 'type'>   

                                                                    1   \
specification                                                  spec001   
extension                                                          png   
filename             p-proj001_s-spec001_d-pizza_v001_c000-001_f000...   
filepath             /tmp/projects/proj001/spec001/pizza/p-proj001_...   
file_error                                                         NaN   
file_traits          {'project': 'proj001', 'specification': 'spec0...   
asset_name                            p-proj001_s-spec001_d-pizza_v001   
asset_path           /tmp/projects/proj001/spec001/pizza/p-proj001_...   
asset_type                                                    sequence   
asset_traits         {'project': ['proj001', 'proj001', 'proj001'],...   
asset_error                                                        NaN   
asset_valid                                                       True   
asset_path_relative                                     <class 'type'>   

                                                                    2   \
specification                                                  spec001   
extension                                                          png   
filename             p-proj001_s-spec001_d-pizza_v001_c000-001_f000...   
filepath             /tmp/projects/proj001/spec001/pizza/p-proj001_...   
file_error                                                         NaN   
file_traits          {'project': 'proj001', 'specification': 'spec0...   
asset_name                            p-proj001_s-spec001_d-pizza_v001   
asset_path           /tmp/projects/proj001/spec001/pizza/p-proj001_...   
asset_type                                                    sequence   
asset_traits         {'project': ['proj001', 'proj001', 'proj001'],...   
asset_error                                                        NaN   
asset_valid                                                       True   
asset_path_relative                                     <class 'type'>   

                                                                    3   \
specification                                                  spec001   
extension                                                          png   
filename             p-proj001_s-spec001_d-pizza_v002_c000-000_f000...   
filepath             /tmp/projects/proj001/spec001/pizza/p-proj001_...   
file_error                                                         NaN   
file_traits          {'project': 'proj001', 'specification': 'spec0...   
asset_name                            p-proj001_s-spec001_d-pizza_v002   
asset_path           /tmp/projects/proj001/spec001/pizza/p-proj001_...   
asset_type                                                    sequence   
asset_traits         {'project': ['proj001', 'proj001', 'proj001', ...   
asset_error                                                        NaN   
asset_valid                                                       Tru

In [6]:
db.search('SELECT * FROM data WHERE version == 3')

project specification descriptor  version  coordinate_x  coordinate_y  \
0  proj001       spec001       kiwi      3.0           0.0           1.0   
1  proj001       spec001      pizza      3.0           0.0           1.0   
2  proj001       spec001      pizza      3.0           0.0           1.0   

   frame extension                                           filename  \
0    1.0       png  p-proj001_s-spec001_d-kiwi_v003_c000-001_f0001...   
1    2.0       png  p-proj001_s-spec001_d-pizza_v003_c000-001_f000...   
2    4.0       png  p-proj001_s-spec001_d-pizza_v003_c000-001_f000...   

                                            filepath  \
0  /tmp/projects/proj001/spec001/pizza/p-proj001_...   
1  /tmp/projects/proj001/spec001/pizza/p-proj001_...   
2  /tmp/projects/proj001/spec001/pizza/p-proj001_...   

                                          file_error  \
0  ValidationError( ['Invalid asset directory nam...   
1                                               None   
2                                               None   

                         asset_name  \
0                              None   
1  p-proj001_s-spec001_d-pizza_v003   
2  p-proj001_s-spec001_d-pizza_v003   

                                          asset_path asset_type  \
0  /tmp/projects/proj001/spec001/pizza/p-proj001_...   sequence   
1  /tmp/projects/proj001/spec001/pizza/p-proj001_...   sequence   
2  /tmp/projects/proj001/spec001/pizza/p-proj001_...   sequence   

                                        asset_error filename_error width  \
0  DataError(\n{'filename_error': 'Rogue field'}\n)           None  None   
1  DataError(\n{'filename_error': 'Rogue field'}\n)           None  None   
2  DataError(\n{'filename_error': 'Rogue field'}\n)           None  None   

  width_error  
0        None  
1        None  
2        None

In [10]:
data = db.data
asset_errors = data[data.file_error.notnull()].copy()
asset_errors['file_errors'] = asset_errors.apply(lambda x: [x.filepath, x.file_error], axis=1)
asset_errors = asset_errors.groupby('asset_path', as_index=False).agg(lambda x: x.tolist())
length = 500
print('-' * length)
for i, row in asset_errors.iterrows():
    print(row.asset_path)
    for item in row.file_errors:
        print(f'\t{item[0]:<150}{item[1]}')
    print('-' * length)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
/tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v003
	/tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v003/misc.txt                                                                         Specification not found in "misc.txt". (at char 0), (line:1, col:1)
	/tmp/projects/proj001/spec001/pizza/p-proj001_s-spec001_d-pizza_v003/p-proj001_s-spec0001_d-pizza_v003_c000-001_f0005.png                             KeyError( Specification not found. )
	/tmp/projects/proj001/